<a href="https://colab.research.google.com/github/thopiyawan/DEMO_rami_dialogflow/blob/master/topic_modeling_th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [40]:
!pwd

/content/drive/.shortcut-targets-by-id/1UHDKxOFjfONnMYU-auICT39cUoR5NF0D/SWU-Master-Thesis-63/Ploy/topicmodeling_thai


In [41]:
cd /content/drive/My Drive/SWU-Master-Thesis-63/Ploy/topicmodeling_thai

/content/drive/.shortcut-targets-by-id/1UHDKxOFjfONnMYU-auICT39cUoR5NF0D/SWU-Master-Thesis-63/Ploy/topicmodeling_thai


In [42]:
import requests
from bs4 import BeautifulSoup
from time import  time, sleep
from random import randint

# from googletrans import Translator

from tqdm import tqdm
from tqdm import tqdm_notebook
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import  SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

In [43]:
dataset = pd.read_csv("dataset/dataset_question_th.csv",encoding='utf8')
print(dataset.shape)
dataset.head()

(9991, 3)


,question,href,date_ask
0,ตั้งครรภ์,https://hd.co.th//ask/27734160,"October 01, 2019 17:30"
1,ตั้งครรภ์5เดือนสามารถตัดปากมดได้ไหม,https://hd.co.th//ask/51006398,"June 30, 2018 14:49"
2,ตั้งครรภ์14สัปดาค่ะปวดท้องเหนือสะดือเเนวยาวร่อ...,https://hd.co.th//ask/4362714,"August 31, 2018 15:07"
3,ตั้งครรภ์24สัปดาห์โดนหมากัดที่ขามีรอยถลอกต้องไ...,https://hd.co.th//ask/33971594,"November 07, 2018 18:58"
4,ตั้งครรภ์ แต่ได๋อัลตราซาวด์ดูหมอบอกมีเลือกออกบ...,https://hd.co.th//ask/48478898,"August 06, 2018 1:35"


In [44]:
print(dataset.shape)
print("===============================================")
print(dataset.info())

(9991, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9991 entries, 0 to 9990
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  9987 non-null   object
 1   href      9990 non-null   object
 2   date_ask  9986 non-null   object
dtypes: object(3)
memory usage: 234.3+ KB
None


In [45]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [46]:
try:
    %tensorflow_version 2.x
except:
    pass

In [47]:
#dev version
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp

!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut

In [48]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
import pythainlp

pythainlp.__version__

'2.2.6'

In [50]:
dataset_q = dataset.iloc[0:50,0]

In [51]:
import re, string, unicodedata

In [52]:
from pythainlp import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import normalize
from pythainlp.spell import correct


stopwords = list(thai_stopwords())


In [53]:
def preprocess_review(reviews): 
    cleaned_words = []
    for review in reviews:
        review = re.sub("[^\u0E00-\u0E7Fa-zA-Z0-9' ]|^'|'$|''", "", str(review))
        words = word_tokenize(review, engine="deepcut")
        tokenized_words = [word for word in words if word not in stopwords]
        # correct_word = correct(str(tokenized_words))
        # lemmatized_word = lemmatize_word(tokenized_words)
        cleaned_words.append(tokenized_words)
    return cleaned_words


In [54]:

# data_np = np.array(str(data_stopw))
data_stopw = preprocess_review(dataset_q)
data_stopw


[['ครรภ์'],
 ['ครรภ์', '5', 'เดือน', 'ตัด', 'ปาก', 'มด', 'ไหม'],
 ['ครรภ์',
  '14',
  'สัปดา',
  'ปวด',
  'ท้อง',
  'เหนือ',
  'สะดือ',
  'เเนวยาว',
  'ร่อง',
  'นม',
  'ทราบ',
  'เปน',
  'ไคำ',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์',
  '24',
  'สัปดาห์',
  'โดน',
  'หมา',
  'กัด',
  'ขา',
  'รอย',
  'ถลอก',
  'หา',
  'หมอ',
  'ไหม'],
 ['ครรภ์',
  ' ',
  'ได๋',
  'อัลตราซาวด์',
  'ดู',
  'หมอ',
  'เลือก',
  'บริเวณ',
  'รก',
  ' ',
  'เรย'],
 ['ครรภ์',
  '5',
  'เดือน',
  'คะมี',
  'ตก',
  'ขาว',
  'สี',
  'น้ำตาล',
  'ขุ่น',
  'แดง',
  'เข้ม',
  'แห้ง',
  'กลิ่น',
  'อาการ',
  'ปกติ',
  'ทราบ',
  'อันตราย',
  'ไหม',
  'ค้ะ',
  'สาเหตุ',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์', '3', 'เดือน', 'ทาน', 'ถาม', 'ตอบ', 'พยาบาล', 'เชี่ยวชาญ'],
 ['ตรวจ',
  'เลือด',
  'ประจำ',
  'ปี',
  'ทำ',
  'งาน',
  ' ',
  'แจ้ง',
  'เลือด',
  ' ',
  'ครรภ์',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์',
  '8',
  'เดือน',
  'ปวด',
  'ฟัน',
  'กราม',
  'ซี่',
  'อุด

In [55]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from multiprocessing import Pool
import nltk
nltk.download('wordnet')

# def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#     def preprocess(text):
#       result = []
#     for token in gensim.utils.simple_preprocess(text):
#         if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
#     return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
dataset = dataset[['question']]
dataset['index'] = dataset.index
dataset = dataset

In [57]:
from sklearn import preprocessing

doc_sample = dataset[dataset['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
      words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(simple_preprocess(doc_sample))

original document: 
['หมอค่ะ', 'คือหนูมีอะไรกับแฟนแล้ว1ครั้ง', 'และอีกวันต่อมามีอะไรกันอีกครั้ง', 'แฟนหลั่งนอกไม่ทันค่ะแต่ถึงกับสุดนะค่ะ', 'และมาหลั่งข้างนอกอีกทีน้ำอุสจิก้มีมากอยู่นะค่ะ', 'มีอะไรติด2วันอ้ะค่ะ', 'วันที่2เชื้ออุสจิจะแข้งแรงพอที่จะท้องได้มั้ยคาะ']


 tokenized and lemmatized document: 
['หมอค', 'อหน', 'อะไรก', 'บแฟนแล', 'คร', 'และอ', 'กว', 'นต', 'อมาม', 'อะไรก', 'นอ', 'กคร', 'แฟนหล', 'งนอกไม', 'นค', 'ะแต', 'งก', 'บส', 'ดนะค', 'และมาหล', 'งข', 'างนอกอ', 'กท', 'ำอ', 'สจ', 'มากอย', 'นะค', 'อะไรต', 'นอ', 'ะค', 'นท', 'เช', 'ออ', 'สจ', 'จะแข', 'งแรงพอท', 'จะท', 'องได', 'ยคาะ']


In [58]:
!pip install pathos

In [59]:

import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
from bson.objectid import ObjectId

import datetime
import calendar
import scipy.stats
from datetime import timedelta, date
import pytz
from pytz import timezone
import multiprocessing
from multiprocessing import cpu_count #For Parallel


cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want to run parallel

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = multiprocessing.Pool(cores)     #สร้าง multiprocessor จำนวนเท่ากับ core cpu ของ server
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

#ตัดคำ
def preprocess(documents):
    list_words= ["สวัสดี","ขอบคุณ","อยู่แล้ว" , "สัปดาห์" ,"หนูก้อรีบ" ,"ชอบเป้น" ,"นิดนึง" ,"ตุลาเลย" , "เหมือน" , "สอบถาม", "อะครับ","คะอยาก","เสี่ยง","ปริมาณ","ต่อเนื่อง","เปื้อน","กางเกง",
                 "สุดท้าย","เรื่อง","รึป่าว","มากน้อย","ชั่วโมง","ประมาณ","บริเวณ","เชี่ยวชาญ","พยาบาล","ประมาน","นิดเดียว","สม่ำเสมอ","กคประจำ","วี่แวว","เที่ยง","คะแล้ว","ลักษณะ",
                 "อาทิตย์","จันทร์","อังคาร","พุธ","พฤหัส","ศุกร์","เสาร์","คะดิฉัน","สองสาม","รุ้ว่า","ปากช่อง","รึเปล่า","สังเกตุ","ทั้งๆที่","เป็นอยู่","มิถุนา","ได้ยิน","แน่นอน","เครื่อง","ตัดสินใจ","อวัยวะ","สมมุติ","เตรียม","ล่วงหน้า","เรียบร้อย","คับคือ",
                 "เมษายน","ธันวาคม","พฤษภาคม","พฤศจิกายน","มกราคม","กุมภาพันธ์","มีนาคม","มิถุนายน","สิงหาคม","กรกฎาคม","ตุลาคม","กันยายน",
                 "จังหวัด", "เลื่อน","ปรึกษา", "เกี่ยว","ต้องการ","รู้สึก","สมบูรณ์","มั่นใจ", "เนื่อง", "สำหรับ", "คะพอดี","รุ้สึก","หาสจาก","ร่างกาย","ห้องน้ำ"]
    result = []
    for token in word_tokenize(documents,engine='deepcut'): 
        token = token.strip()
        if token not in stopwords and len(token) >5 and token not in list_words:
            token = correct(token)
            result.append(lemmatize_stemming(token))
    return result


def func(documents):
    res = documents['headline_text'].map(preprocess)
    return res

def cleanText(documents):
    documents['headline_text'] = documents['headline_text'].str.replace('|',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('\\',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('\/',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('.',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('_','')
    documents['headline_text'] = documents['headline_text'].str.replace('\d+', '')
    documents['headline_text'] = documents['headline_text'].str.replace('-',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('+',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('+',' ')
    pattern = '|'.join(['&', '%', ';', '='])
    documents['headline_text'] = documents['headline_text'].str.replace(pattern, ' ')
    # documents = documents.loc[~documents['headline_text'].str.contains("img")]
    documents.headline_text.replace({r"[^ก-๙]":''}, regex=True, inplace=True)
    documents['headline_text'] = documents['headline_text'].str.replace(':',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('#',' ')
    documents['headline_text'] = documents['headline_text'].str.strip()
    documents = documents.loc[documents['headline_text']!= ""]
    documents = documents.dropna().reset_index(drop=True)
    return documents


In [60]:
# documents = dataset.copy()
# documents = documents.rename(columns={'question':'headline_text'})

# # concatenate all string of same user to be headline_text
# documents = documents.reset_index()
# print(documents.shape[0])
# documents.head()
dataset = dataset.iloc[:,:]
documents = dataset.copy()
documents = documents.rename(columns={'question':'headline_text'})

# concatenate all string of same user to be headline_text
# documents = documents.agg(lambda col: ' '.join(col)).reset_index()
documents = documents.groupby(['headline_text'], as_index = False).agg({'headline_text': lambda x: ' '.join(x.astype(str))})
print(documents.shape[0])
documents.head()




9891


,headline_text
0,1 ถ้าฝังยาคุมแล้วหมดอายุ ไม่เอายาคุมออกได้ไม๊ค...
1,1 ถ้าเรากินยาคุมหมดแล้วประจำเดือนมาแล้ว ไม่กิน...
2,1 ทานยาคุมมาตั้งแต่29เดือนกันยาวันที่ประจำเดือ...
3,1 พอดีเมื่อคืนนี้มีอะไรกับแฟนแล้วถุงแตกเผลอแตก...
4,1 มีอาการปัสสาวะเล็ดบ่อย บางครั้งก็ไม่รู้ตัวค่...


In [61]:
documents = parallelize(documents,cleanText)
documents.head()


,headline_text
0,ถ้าฝังยาคุมแล้วหมดอายุไม่เอายาคุมออกได้ไม๊คะจะ...
1,ถ้าเรากินยาคุมหมดแล้วประจำเดือนมาแล้วไม่กินต่อ...
2,ทานยาคุมมาตั้งแต่เดือนกันยาวันที่ประจำเดือนมาว...
3,พอดีเมื่อคืนนี้มีอะไรกับแฟนแล้วถุงแตกเผลอแตกใน...
4,มีอาการปัสสาวะเล็ดบ่อยบางครั้งก็ไม่รู้ตัวค่ะถ้...


In [62]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from pythainlp.tokenize import word_tokenize


allLen = documents.shape[0]
progress = 0
# Print iterations progress
def printProgressBar (iteration, total,mode = 0,current_percent = 0, prefix = '', suffix = '', decimals = 1, length = 100, fill = ' '):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        mode        - Optional  : Mode of output (Int : (dafault 0 : grahical progress bar , 1 : standard text))
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    if mode == 0 :
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
        # Print New Line on Complete
        if iteration == total:
            print()
    elif mode == 1:
        if current_percent != round(float(percent),1):
            current_percent = round(float(percent),1)
            print(percent+"%")
        return current_percent
    return 0

In [63]:
!pip install pytesseract

In [64]:
stemmer = PorterStemmer()


In [65]:

processed_docs = documents['headline_text'].map(preprocess)

In [66]:
processed_docs

0                                    []
1                            [สัมพันธ์]
2       [เพศสัมพันธ์, ฉุกเฉิน, ฉุกเฉิน]
3                [คัดเต้า, เพศสัมพันธ์]
4                [ปัสสาวะ, เพศสัมพันธ์]
                     ...               
4940                           [สอดใส่]
4941                      [เพศสัมพันธ์]
4942                                 []
4943                                 []
4944                                 []
Name: headline_text, Length: 9888, dtype: object

In [67]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 สัมพันธ์
1 ฉุกเฉิน
2 เพศสัมพันธ์
3 คัดเต้า
4 ปัสสาวะ
5 ยี่ห้อ
6 เมโลดี้
7 ถ่ายเหลว
8 ป้องกัน
9 เจลล่อลื่น
10 อันตราย


In [68]:
dictionary

In [69]:
# ตัดคำฟุ่มเฟือยออก โดยลบคำที่ปรากฏบ่อยเกิน 50% และให้เหลือไปถึง1พันคำ
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)  #reduce to 1K word


# Gensim doc2bow
# For each document we create a dictionary reporting how many
# words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

# run LDA แบ่งคำออกเป็น9กลุ่ม โดยใช้ cpu ทุกตัว (serverที่เราใช้มี32cpu)
#worker = cpu core amount -1
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=31)

# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

[]
Topic: 0 
Words: 0.323*"ฉุกเฉิน" + 0.235*"เพศสัมพันธ์" + 0.074*"น้ำตาล" + 0.052*"ป้องกัน" + 0.038*"กำเนิด" + 0.023*"รับประทาน" + 0.017*"เบาหวาน" + 0.016*"เครียด" + 0.015*"สิทธิ์" + 0.010*"สุขภาพ"
Topic: 1 
Words: 0.102*"อันตราย" + 0.097*"ฉุกเฉิน" + 0.093*"หล่อลื่น" + 0.083*"เครียด" + 0.060*"กำเนิด" + 0.052*"ป้องกัน" + 0.044*"ผ่าตัด" + 0.033*"ติดต่อ" + 0.032*"สอดใส่" + 0.029*"เพศสัมพันธ์"
Topic: 2 
Words: 0.132*"ใช้จ่าย" + 0.087*"ฉุกเฉิน" + 0.081*"เพศสัมพัน" + 0.048*"เพศสัมพันธ์" + 0.044*"อนามัย" + 0.036*"ตั้งครรภ์" + 0.033*"เครียด" + 0.030*"เพื่อน" + 0.030*"อัลตราซาวนด์" + 0.028*"สัมพันธ์"
Topic: 3 
Words: 0.101*"เพศสัมพันธ์" + 0.097*"ฉุกเฉิน" + 0.068*"กำเนิด" + 0.061*"วัคซีน" + 0.044*"มะเร็ง" + 0.043*"ตั้งท้อง" + 0.041*"เมอซิลอน" + 0.041*"ตามิน" + 0.034*"ท้องน้อย" + 0.034*"สอดใส่"
Topic: 4 
Words: 0.316*"เพศสัมพันธ์" + 0.080*"สาเหตุ" + 0.064*"อาเจียน" + 0.047*"กำเนิด" + 0.046*"อันตราย" + 0.044*"น้ำหนัก" + 0.033*"ประสิทธิภาพ" + 0.025*"ฉุกเฉิน" + 0.022*"คลื่นไส้" + 0.019*"สอดใส่"
Top

In [70]:
unseen_document = 'เป็นเบาหวานจะมีผลกระทบกับการตั้งครรภ์'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 15)))

Score: 0.10000000149011612	 Topic: 0.323*"ฉุกเฉิน" + 0.235*"เพศสัมพันธ์" + 0.074*"น้ำตาล" + 0.052*"ป้องกัน" + 0.038*"กำเนิด" + 0.023*"รับประทาน" + 0.017*"เบาหวาน" + 0.016*"เครียด" + 0.015*"สิทธิ์" + 0.010*"สุขภาพ" + 0.009*"เพศสัมพัน" + 0.008*"เปอร์เซ็นต์" + 0.008*"สัมพันธ์" + 0.007*"โรงพยาบาล" + 0.006*"กระดาษ"
Score: 0.10000000149011612	 Topic: 0.102*"อันตราย" + 0.097*"ฉุกเฉิน" + 0.093*"หล่อลื่น" + 0.083*"เครียด" + 0.060*"กำเนิด" + 0.052*"ป้องกัน" + 0.044*"ผ่าตัด" + 0.033*"ติดต่อ" + 0.032*"สอดใส่" + 0.029*"เพศสัมพันธ์" + 0.027*"แบบแผน" + 0.025*"อักเสบ" + 0.022*"วิตามิน" + 0.021*"เพศสัมพันธุ์" + 0.017*"เหลือง"
Score: 0.10000000149011612	 Topic: 0.132*"ใช้จ่าย" + 0.087*"ฉุกเฉิน" + 0.081*"เพศสัมพัน" + 0.048*"เพศสัมพันธ์" + 0.044*"อนามัย" + 0.036*"ตั้งครรภ์" + 0.033*"เครียด" + 0.030*"เพื่อน" + 0.030*"อัลตราซาวนด์" + 0.028*"สัมพันธ์" + 0.028*"รับประทาน" + 0.022*"อันตราย" + 0.022*"น้ำตาล" + 0.022*"คัดเต้า" + 0.019*"หน้าอก"
Score: 0.10000000149011612	 Topic: 0.101*"เพศสัมพันธ์" + 0.097*"ฉุกเฉ

In [71]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Dictionary for each document
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# print(bow_corpus[4310])

# Applying TF-IDF
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
# for doc in corpus_tfidf:
#     pprint(doc)
#     break

# Train corpus with LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary,
                                       passes=2, workers=2)
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

# LDA with TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary,
                                             passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

# Performance evaluation of LDA with Bag of Words
# for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

# Performance Evaluation of LDA with TF-IDF
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

# Testing
unseen_document = 'จะท้องไหมคะ'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Topic: 0 Word: 0.447*"เพศสัมพันธ์" + 0.146*"ฉุกเฉิน" + 0.089*"ป้องกัน" + 0.020*"ยี่ห้อ" + 0.018*"หายขาด" + 0.017*"กำเนิด" + 0.017*"เดือน" + 0.017*"สอดใส่" + 0.016*"ประจำ" + 0.016*"มะเร็ง"
Topic: 1 Word: 0.346*"กำเนิด" + 0.129*"อนามัย" + 0.070*"ป้องกัน" + 0.053*"เพศสัมพันธุ์" + 0.032*"เหลือง" + 0.028*"ข้อมูล" + 0.026*"เพื่อน" + 0.023*"ธรรมชาติ" + 0.021*"เหนื่อย" + 0.020*"เพศสัมพันธ์"
Topic: 2 Word: 0.157*"สัมพันธ์" + 0.147*"ใช้จ่าย" + 0.123*"แป้งมัน" + 0.087*"ฮอร์โมน" + 0.060*"สอดใส่" + 0.047*"อาเจียน" + 0.035*"แต่งงาน" + 0.029*"คลินิก" + 0.027*"ไทรอยด์" + 0.022*"หล่อลื่น"
Topic: 3 Word: 0.135*"อันตราย" + 0.122*"ปัสสาวะ" + 0.083*"ผ่าตัด" + 0.067*"หล่อลื่น" + 0.066*"รับประทาน" + 0.050*"แป้งมัน" + 0.039*"กะปริบกะปรอย" + 0.037*"คลื่นไส้" + 0.033*"ฉุกเฉิน" + 0.030*"ของแข็ง"
Topic: 4 Word: 0.118*"สาเหตุ" + 0.109*"ยี่ห้อ" + 0.105*"สิทธิ์" + 0.073*"น้ำตาล" + 0.067*"ติดต่อ" + 0.058*"เครียด" + 0.034*"ท้องที่" + 0.033*"กำเนิด" + 0.025*"เพศสัมพันธ์" + 0.025*"ประเด็น"
Topic: 5 Word: 0.202*"ป้องกัน"

In [72]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]


[(0,
  '0.403*"กำเนิด" + 0.114*"สอดใส่" + 0.076*"ใช้จ่าย" + 0.067*"เพศสัมพันธ์" + '
  '0.026*"สัมพันธ์" + 0.021*"กะปริบกะปรอย" + 0.018*"สิทธิ์" + 0.017*"ฉุกเฉิน" '
  '+ 0.016*"ประกัน" + 0.015*"ป้องกัน"'),
 (1,
  '0.250*"สัมพันธ์" + 0.128*"เพศสัมพันธ์" + 0.106*"ป้องกัน" + 0.076*"ฉุกเฉิน" '
  '+ 0.047*"เพศสัมพัน" + 0.045*"หลั่งนอก" + 0.027*"เครียด" + 0.026*"คัดเต้า" + '
  '0.018*"ตั้งท้อง" + 0.018*"หน้าอก"'),
 (2,
  '0.273*"อันตราย" + 0.126*"รับประทาน" + 0.058*"อักเสบ" + 0.043*"เพศสัมพันธ์" '
  '+ 0.039*"ป้องกัน" + 0.035*"ท้องน้อย" + 0.027*"มะเร็ง" + 0.023*"เพสสัมพัน" + '
  '0.022*"ข้อมูล" + 0.022*"อารมณ์"'),
 (3,
  '0.154*"ฮอร์โมน" + 0.100*"เครียด" + 0.085*"เพศสัมพันธ์" + 0.082*"อาเจียน" + '
  '0.080*"สิทธิ์" + 0.049*"ตั้งท้อง" + 0.049*"เมอซิลอน" + 0.041*"สุขภาพ" + '
  '0.033*"เหนื่อย" + 0.026*"ตั้งใจ"'),
 (4,
  '0.323*"เพศสัมพันธ์" + 0.116*"ฉุกเฉิน" + 0.106*"น้ำตาล" + 0.057*"ผ่าตัด" + '
  '0.033*"แต่งงาน" + 0.031*"กำเนิด" + 0.021*"เคลื่อน" + 0.020*"หายขาด" + '
  '0.020*"อนุญาต" + 0.019

**============================================================================**

In [73]:
!pip3 install spacy
!python3 -m spacy download en #Language model
!pip3 install gensim # For topic modeling
!pip3 install pyLDAvis # For visualizing topic models

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [74]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.447146089283022

Coherence Score:  0.5623758434479108


Output:
- Lower the perplexity better the model.
- Higher the topic coherence, the topic is more human interpretable.

In [76]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.183410 -0.034283       1        1  14.916886
8     -0.131377  0.135049       2        1  13.619134
0      0.057104 -0.157017       3        1  11.746269
1     -0.146690  0.003853       4        1  10.245668
6      0.055071 -0.189945       5        1   9.346443
4     -0.122652  0.067053       6        1   9.055216
3      0.088693  0.075918       7        1   8.671699
2      0.130478  0.197692       8        1   7.690259
5      0.272610  0.017758       9        1   7.666308
7     -0.019827 -0.116079      10        1   7.042118, topic_info=        Term         Freq        Total Category  logprob  loglift
1    ฉุกเฉิน  1121.000000  1121.000000  Default  30.0000  30.0000
18    กำเนิด   805.000000   805.000000  Default  29.0000  29.0000
8    อันตราย   311.000000   311.000000  Default  28.0000  28.0000
0   สัมพันธ์   375.000000   375.000000  Default  27.0000  27.0000
5     ยี่ห้อ   309.000000   309.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
64   แป้งมัน     7.602131   183.767349  Topic10  -4.7091  -0.5320
7    ป้องกัน    14.614365   738.815104  Topic10  -4.0555  -1.2698
35    ติดต่อ     4.386956   105.558566  Topic10  -5.2588  -0.5274
70    น้ำตาล     4.955352   204.895183  Topic10  -5.1370  -1.0688
28    สอดใส่     3.483639   234.740662  Topic10  -5.4894  -1.5571

[406 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
119       2  0.056506   กระดาษ
119       8  0.791077   กระดาษ
119      10  0.113011   กระดาษ
99        2  0.140077  กระเพาะ
99        9  0.747079  กระเพาะ
...     ...       ...      ...
109       1  0.131178   ไมเกรน
109       2  0.721480   ไมเกรน
109       3  0.032795   ไมเกรน
109       7  0.032795   ไมเกรน
109      10  0.032795   ไมเกรน

[719 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 1, 2, 7, 5, 4, 3, 6, 8])

bubble แต่ละอันคือ topic <br>
bubble ที่ใหญ่จะมีความแพร่หลายและโดดเด่นมาก <br>

รูปแบบหัวข้อที่ดีจะเป็นหัวข้อที่ค่อนข้างใหญ่ซึ่งกระจัดกระจายอยู่ในควอดแดรนต์ที่แตกต่างกันแทนที่จะกระจุกอยู่ในจตุภาคเดียว  <br>

- The model with too many topics will have many overlaps, small sized bubbles clustered in one region of chart.
- If you move the cursor the different bubbles you can see different keywords associated with topics.